In [1]:
import pandas as pd
import os

global ctl
global hyb
global ast
global esc

In [2]:
#
#
#
#
def dictSetup():
    team = dict()

    team["Atlanta Reign"] = "ATL"
    team["Boston Uprising"] = "BOS"
    team["Chengdu Hunters"] = "CHD"
    team["Dallas Fuel"] = "DAL"
    team["Florida Mayhem"] = "FLO"
    team["Guangzhou Charge"] = "GZC"
    team["Hangzhou Spark"] = "HZS"
    team["Houston Outlaws"] = "HOU"
    team["London Spitfire"] = "LDN"
    team["Los Angeles Gladiators"] = "GLA"
    team["Los Angeles Valiant"] = "VAL"
    team["New York Excelsior"] = "NYE"
    team["Paris Eternal"] = "PAR"
    team["Philadelphia Fusion"] = "PHI"
    team["San Francisco Shock"] = "SFS"
    team["Seoul Dynasty"] = "SEO"
    team["Shanghai Dragons"] = "SHD"
    team["Toronto Defiant"] = "TOR"
    team["Vancouver Titans"] = "VAN"
    team["Washington Justice"] = "WAS"

    mp = dict()

    mp["Ilios"] = ctl
    mp["Busan"] = ctl
    mp["Nepal"] = ctl
    mp["Oasis"] = ctl
    mp["Lijiang Tower"] = ctl

    mp["Havana"] = esc
    mp["Rialto"] = esc
    mp["Watchpoint: Gibraltar"] = esc
    mp["Dorado"] = esc
    mp["Junkertown"] = esc
    mp["Route 66"] = esc

    mp["Horizon Lunar Colony"] = ast
    mp["Hanamura"] = ast
    mp["Temple of Anubis"] = ast
    mp["Volskaya Industries"] = ast

    mp["Eichenwalde"] = hyb
    mp["King's Row"] = hyb
    mp["Blizzard World"] = hyb
    mp["Numbani"] = hyb
    mp["Hollywood"] = hyb


In [4]:
# set up elo files. only the latest data is ouputed as dictionary
# 
# input:  filename/path to file containing elo data. default is ELO.csv
# output: 2-tuple containing 1) dataframe of the entire elo history and 
#             2) dictionary containing the latest elo values
def setup(file = "ELO.csv"):
    rating = pd.read_csv( file)
    dictSetup()
    return (rating,dict(rating.iloc[-1,:]))

In [5]:
# 
#
# input:  
# output: updated elo ratings
def updateELO(rating, teamA, teamB, sA, sB):
    if (sA > sB):
        sA = 1
    elif( sA < sB):
        sA = 0
    else:
        sA = .5
    sB = 1 - sA
    
    qA = 10**(rating[teamA] / 400)
    qB = 10**(rating[teamB] / 400)
    
    eA = qA/(qA + qB)
    eB = qB/(qA + qB)
    
    rating[teamA] = rating[teamA] + 20 * (sA - eA)
    rating[teamB] = rating[teamB] + 20 * (sB - eB)
    
    return rating

In [25]:
x,rating = setup()

control,ctl = setup("ControlELO.csv")
hybrid,hyb = setup("HybridELO.csv")
assault,ast = setup("AssaultELO.csv")
escort,esc = setup("EscortELO.csv")

df = pd.read_csv("match_map_stats.csv")

del df['round_start_time']
del df['stage']

del df['control_round_name']
del df['attacker']
del df['defender']
del df['team_one_name']
del df['team_two_name']
del df['attacker_payload_distance']
del df['defender_payload_distance']
del df['attacker_time_banked']
del df['defender_time_banked']
del df['attacker_control_perecent']
del df['defender_control_perecent']
del df['attacker_round_end_score'] 
del df['defender_round_end_score']

rows, c = df.shape
mid =-1

for i in range(rows):
    row = df.iloc[i,:]
    
    if i == 0:
        mid = row[1]
        gn = row[2]
        
        mp[row[6]] = updateELO(mp[row[6]],row[4],row[5],row[8],row[9])
    print(row[1])
    if mid != row[1]:
        mid = row[1]


10223


NameError: name 'mid' is not defined

In [ ]:
#
#
# input:  the four updated map elos
# output: files updated with latest elo data
def saveELO(control, hybrid, assault, escort):
    stELO = ""    
    stCTL = ""  
    stHYB = ""  
    stAST = ""  
    stESC = ""  
    
    tmp = control
    
    for key in tmp:
        stCTL = stCTL + "," + str(control[key])
        stHYB = stHYB + "," + str(hybrid[key])
        stAST = stAST + "," + str(assault[key])
        stESC = stESC + "," + str(escort[key])
        
        tmp[key] = control[key] + hybrid[key] + assault[key] + escort[key]
        tmp[key] = tmp[key]/4
        
        stELO = stELO + "," + str(tmp[key])
    
    #print(stCTL[1:])
    #print(stHYB[1:])
    #print(stAST[1:])
    #print(stESC[1:])
    #print(stELO[1:])
    open("ControlELO.csv", "a").write(stCTL[1:]).close()
    open("HybridELO.csv", "a").write(stHYB[1:]).close()
    open("AssaultELO.csv", "a").write(stAST[1:]).close()
    open("EscortELO.csv", "a").write(stESC[1:]).close()
    open("ELO.csv", "a").write(stELO[1:]).close()